In [1]:
from PIL import Image
import numpy as np
import os
import matplotlib.pyplot as plt
from io import BytesIO
import requests as req

def read_image(imageName):
    im = Image.open(imageName).convert('RGB')
    data = np.array(im)
    return data

def read_image_url(img_url):
    response = req.get('http://'+img_url)
    im = Image.open(BytesIO(response.content)).convert('RGB')
    data = np.array(im)
    return data

def get_num(n):
    l_num = []
    m = 0
    x = n
    while True:
        if m < 9:
            x0,x1, = np.split(x, [30],axis = 1)
            #print(x0)
            #plt.imshow(x0)
            #plt.pause(0.001)
            #print(x1)
            #plt.imshow(x1)
            #plt.pause(0.001)
            l_num.append(x0)
            x = x1
            m+=1
            continue
        else:
            l_num.append(x)
            #plt.imshow(x)
            #plt.pause(0.001)
            break
    return l_num

def get_0_9():
    l_num = get_num(read_image('test001.png'))
    x2,x4,x3,x6,x8,x5,x1,x9,x0,x7 = l_num
    list_num = [x0,x1,x2,x3,x4,x5,x6,x7,x8,x9]
    return list_num

def xx1(png):
    l = []
    list_num = get_0_9()
    l_num_test = get_num(read_image_url(png))
    for i in range(0,10):
        #plt.imshow(l_num_test[i])
        #plt.pause(0.001)
        for w in range(0,10):
            #print((i == w).all())
            if (l_num_test[i] == list_num[w]).all():
                #plt.imshow(w)
                #plt.pause(0.001)
                l.append(w)
                #print(w)
                break
    #print (l)
    return l




In [2]:
a = xx1('static8.ziroom.com/phoenix/pc/images/price/e72ac241b410eac63a652dc1349521fds.png')
print(a)

[4, 0, 1, 5, 9, 8, 3, 7, 6, 2]


In [3]:
import urllib.request
import urllib.parse
from bs4 import BeautifulSoup
import re
import csv
import codecs

def get_i(url):
    #url = "http://www.ziroom.com/z/nl/z3.html?p=2"
    request_all = urllib.request.Request(url)
    request_all.add_header('User-Agent','Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.1.6) Gecko/20091201 Firefox/3.5.6')
    opener_all = urllib.request.build_opener()
    web_all = opener_all.open(request_all)
    html_all = web_all.read()
    soup_all = BeautifulSoup(html_all,'lxml')
    soup_all_texts = soup_all.find('ul',id='houseList')
    #soup_page_texts = soup_all_texts.find_all('li',class_='clearfix')
    a = soup_all_texts.find_all('div',class_='txt')
    #print(len(a))
    room_list = []
    
    room_price = get_price(url)
    for i in range(0,len(a)):
        room_one = []
        room = a[i]
        #print(room)
        room_name = room.find('a',class_='t1').string
        room_url = room.find('a')['href']
        room_url = re.sub('//', '', room_url)
        #print(room_url)
        room_i_span = room.find_all('p')
        room_price_one = room_price[i]
        #print(len(room_i_span))
        for ii in (0,1):
            if ii == 0:
                room_i_span_0 = room_i_span[0]
                room_i_mianji = room_i_span_0.select('span')[0].get_text().strip()
                room_i_louceng = room_i_span_0.select('span')[1].get_text().strip()
                room_i_huxing = room_i_span_0.select('span')[2].get_text().strip()
            if ii == 1:
                room_i_span_1 = room_i_span[1]
                room_i_ditie = room_i_span_1.select('span')[0].get_text().strip()
        #print (room_name)
        #print (room_i_mianji,' ',room_i_louceng,' ',room_i_huxing,' ',room_i_ditie)
        #print (room_i_span)
        room_one.append(room_name)
        room_one.append(room_i_mianji)
        room_one.append(room_i_louceng)
        room_one.append(room_i_huxing)
        room_one.append(room_i_ditie)
        room_one.append(room_price_one)
        room_one.append(room_url)
        room_list.append(room_one)
    return room_list
    #print (a)
    
def get_price(url):    
    request_all = urllib.request.Request(url)
    request_all.add_header('User-Agent','Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.1.6) Gecko/20091201 Firefox/3.5.6')
    opener_all = urllib.request.build_opener()
    web_all = opener_all.open(request_all)
    html_all = web_all.read()
    soup_all = BeautifulSoup(html_all,'lxml')
    soup_all_texts = soup_all.find_all('script')
    m_pic = str(soup_all_texts[5])
    #print (m_pic)
    key = r'//.+png'
    png = str(re.findall(key,m_pic))
    png = re.sub('//', '', png)
    png = re.sub('\[', '', png)
    png = re.sub('\]', '', png)
    png = png.strip("'")
    #print(png)
    key1 = r'\[\[.+\]\]'
    var_preice = re.findall(key1,m_pic)
    v = var_preice[0]
    #print(v)
    '''
    v = re.sub('[\[\]]', '', v)
    #print (v)
    l_v = list(v.split(","))
    print (l_v)
    list_price = [l_v[i:i+4] for i in range(0,len(l_v),4)]
    #print (list_price)
    '''
    list_price = get_price_num(v)
    #print (list_peice)
    real_price = xx1(png)
    #print(real_price)
    l_1=[]
    for i in range(0,len(list_price)):
        a = list_price[i]
        ln = []
        for ii in range(0,len(a)):
            b = real_price[int(a[ii])]
            #print(b)
            ln.append(str(b))
        s=''.join(ln)
        l_1.append(s)
    #print (l_1)
    return l_1

def csv_save(room_list,x):
    csv_name = 'page'
    with open(csv_name+str(x)+'.csv', 'w', newline='', encoding='utf-8') as csv_file:
        csv_writer = csv.writer(csv_file)
        for i in range(0,len(room_list)):
            csv_writer.writerow(room_list[i])

def get_page_list(url,num):
    #url = "http://www.ziroom.com/z/nl/z2-o4.html?p="
    page_list = []
    for i in range(1,num+1):
        try:
            new_url = url+str(i)
            page_list.append(new_url)
        except:
            pass
    return page_list

def get_price_num(str_price):
    var_preice = re.split('\W+,', str_price)
    #print (var_preice)
    l_111 = []
    for x in range(0,len(var_preice)):
        v = var_preice[x] 
        v = re.sub('[\[\]]', '', v)
        #print (v)
        v_1 = v.split(",")
        l_111.append(v_1)
        #print(l_111)
    return(l_111)
    #print (l_111)

def get_url_list():
    url = 'http://www.ziroom.com/z/nl/z2-o4.html'
    request_all = urllib.request.Request(url)
    request_all.add_header('User-Agent','Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.1.6) Gecko/20091201 Firefox/3.5.6')
    opener_all = urllib.request.build_opener()
    web_all = opener_all.open(request_all)
    html_all = web_all.read()
    soup_all = BeautifulSoup(html_all,'lxml')
    soup_all_texts = soup_all.find('ul', class_='clearfix filterList')
    a = soup_all_texts.find_all('li')
    #print(soup_all_texts)
    l_url_list = []
    for i in range(1,len(a)):
        l_url_one = []
        filter_name = a[i].find('a').string
        b = a[i].find('a')['href']
        b = re.sub('//', '', b)
        b = 'http://'+b
        page_all_num = get_page_allnum(b)
        b = b+'?p='
        l_url_one.append(filter_name)
        l_url_one.append(b)
        l_url_one.append(page_all_num)
        l_url_list.append(l_url_one)
    return l_url_list

def get_page_allnum(url):
    request_all = urllib.request.Request(url)
    request_all.add_header('User-Agent','Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.1.6) Gecko/20091201 Firefox/3.5.6')
    opener_all = urllib.request.build_opener()
    web_all = opener_all.open(request_all)
    html_all = web_all.read()
    soup_all = BeautifulSoup(html_all,'lxml')
    page = soup_all.find('span', class_='pagenum').get_text().strip()
    #print (page)
    page = re.sub('1/','',page)
    return int(page)

In [4]:
print(get_url_list())


[['东城', 'http://www.ziroom.com/z/nl/z2-o4-d23008614.html?p=', 18], ['西城', 'http://www.ziroom.com/z/nl/z2-o4-d23008626.html?p=', 35], ['朝阳', 'http://www.ziroom.com/z/nl/z2-o4-d23008613.html?p=', 50], ['海淀', 'http://www.ziroom.com/z/nl/z2-o4-d23008618.html?p=', 50], ['丰台', 'http://www.ziroom.com/z/nl/z2-o4-d23008617.html?p=', 50], ['石景山', 'http://www.ziroom.com/z/nl/z2-o4-d23008623.html?p=', 27], ['通州', 'http://www.ziroom.com/z/nl/z2-o4-d23008625.html?p=', 50], ['昌平', 'http://www.ziroom.com/z/nl/z2-o4-d23008611.html?p=', 50], ['大兴', 'http://www.ziroom.com/z/nl/z2-o4-d23008615.html?p=', 50], ['顺义', 'http://www.ziroom.com/z/nl/z2-o4-d23008624.html?p=', 50], ['房山', 'http://www.ziroom.com/z/nl/z2-o4-d23008616.html?p=', 24], ['门头沟', 'http://www.ziroom.com/z/nl/z2-o4-d23008620.html?p=', 9], ['亦庄开发区', 'http://www.ziroom.com/z/nl/z2-o4-d23008629.html?p=', 9]]


In [11]:
def get_csvdata(i,url_list):
    #url_list = get_url_list()
    #for i in range(0,1):
    #for i in range(0,len(url_list)):
    area_name = url_list[i][0]
    print(area_name)
    area_url = url_list[i][1]
    area_url_num = url_list[i][2]
    page_list = get_page_list(area_url,area_url_num)
    #print(page_list)
    room_list_allpage = []
    for i in range(0,len(page_list)):
        print(('page : %d') %(i) )
        url = page_list[i]
        room_list = get_i(url)
        room_list_allpage.extend(room_list)
    #print(room_list_allpage)
    return area_name,room_list_allpage
        

In [12]:
import pandas as pd
from pandas.core.frame import DataFrame

url_list = get_url_list()
for i in range (0,len(url_list)):
    area_name,room_list_allpage = get_csvdata(i,url_list)
    data=DataFrame(room_list_allpage)
    df = pd.DataFrame(data)    #colums=['房屋','面积','楼层','户型','位置','价格','网址']
    df.columns = ['房屋','面积','楼层','户型','位置','价格','网址']
    #df
    df.to_csv(area_name+'.csv') #不保存列名


东城
page : 0
page : 1
page : 2
page : 3
page : 4
page : 5
page : 6
page : 7
page : 8
page : 9
page : 10
page : 11
page : 12
page : 13
page : 14
page : 15
page : 16
page : 17
西城
page : 0
page : 1
page : 2
page : 3
page : 4
page : 5
page : 6
page : 7
page : 8
page : 9
page : 10
page : 11
page : 12
page : 13
page : 14
page : 15
page : 16
page : 17
page : 18
page : 19
page : 20
page : 21
page : 22
page : 23
page : 24
page : 25
page : 26
page : 27
page : 28
page : 29
page : 30
page : 31
page : 32
page : 33
page : 34
朝阳
page : 0
page : 1
page : 2
page : 3
page : 4
page : 5
page : 6
page : 7
page : 8
page : 9
page : 10
page : 11
page : 12
page : 13
page : 14
page : 15
page : 16
page : 17
page : 18
page : 19
page : 20
page : 21
page : 22
page : 23
page : 24
page : 25
page : 26
page : 27
page : 28
page : 29
page : 30
page : 31
page : 32
page : 33
page : 34
page : 35
page : 36
page : 37
page : 38
page : 39
page : 40
page : 41
page : 42
page : 43
page : 44
page : 45
page : 46
page : 47
page : 48
p

In [25]:
df_test=pd.read_csv('./room_list/utf-8/dongcheng.csv')#encoding='utf-8'
df_test

,Unnamed: 0,房屋,面积,楼层,户型,位置,价格,网址
0,0,友家 · 东花市北里中区4居室-南卧,27.12 ㎡,6/6层,4室1厅,距7号线广渠门内站直线655米,4890,www.ziroom.com/z/vr/61965816.html
1,1,友家 · 新奥洋房3居室-南卧,约25 ㎡,10/11层,3室1厅,距14号线景泰站直线173米,3990,www.ziroom.com/z/vr/62087543.html
2,2,友家 · 胜古中路1号院4居室-东卧,22 ㎡,3/14层,4室1厅,距5号线和平西桥站直线593米,5390,www.ziroom.com/z/vr/60653794.html
3,3,友家 · 安德里北街25号院2居室-西北卧,18.2 ㎡,10/17层,2室1厅,距8号线鼓楼大街站直线933米,4060,www.ziroom.com/z/vr/60045911.html
4,4,友家 · 东直门北大街2居室-东卧,约18.1 ㎡,8/12层,2室1厅,距2号线东直门站直线229米,4160,www.ziroom.com/z/vr/197686.html
5,5,友家 · 海运仓小区3居室-东卧,约18 ㎡,1/6层,3室1厅,距2号线东四十条站直线441米,139,www.ziroom.com/z/vr/60849.html
6,6,友家 · 安外西河沿2居室-南卧,约18 ㎡,5/6层,2室1厅,距2号线安定门站直线190米,4390,www.ziroom.com/z/vr/62075909.html
7,7,友家 · 青年湖北里2居室-南卧,约18 ㎡,4/5层,2室1厅,距2号线安定门站直线980米,4290,www.ziroom.com/z/vr/62085475.html
8,8,友家 · 富贵园三区3居室-东卧,约17.8 ㎡,9/18层,3室1厅,距7号线广渠门内站直线198米,129,www.ziroom.com/z/vr/60101754.html
9,9,友家 · 花市枣苑二期4居室-西卧,17.5 ㎡,7/17层,4室1厅,距7号线广渠门内站直线405米,4190,www.ziroom.com/z/vr/61634441.html
